origin: http://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html  
translator: Hongpu Liu

In [2]:
%matplotlib inline
from __future__ import print_function, division

# 进阶：制定动态决策和双向LSTM与CRF
## 1. 动态 vs 静态深度学习工具
Pytorch是一个动态神经网络工具。另一个动态工具是**DyNet**（这里提到DyNet是因为它的工作方式与PyTorch相似，如果你看了一个DyNet的例子，那么对你在PyTorch中实现它会有帮助）。相反的例子是**静态**工具集，例如*Theano*，*Keras*，*TensorFlow*等等。其核心的差别在于：
- 在静态工具中，定义计算图一次，然后编译。然后将样本传递给它。
- 在动态工具中，为每个实例定义一个计算图，即时运行无需编译。

没有很多的经验，很难欣赏其中的差异。例如，假设我们要构建一个深度句法解析器，我们的模型大致涉及以下步骤：
- 自底向上构建树
- 标注根节点（句子中的词）
- 然后，利用神经网络和词嵌入来寻找句法形式的组合。当我们形成一个新的句法，利用某种技术来获得句法的嵌入。在这种情况下，我们的网络结构将完全依赖输入的句子。在句子“The green cat scratched the wall”中，在模型的某处，我们需要组合跨度$(i,j,r)=(1,3,NP)$（句法NP跨越第一个词到第三个词，在本例中是“The green cat”）。

然而，另一个句子可能是“Somewhere, the big fat cat scratched the wall”。在这个句子中，我们需要在某处形成句法$(2,4,NP)$。我们想要形成的句法依赖于样本实例。在静态工具中，我们只编译计算图一次，这将非常困难甚至无法编写这一逻辑。在动态工具集中，我们不是只有一个预定义的计算图，可以为每一个新的样本实例构建新的计算图。

动态工具集还有易于调试、代码更接近宿主语言（PyTorch和DyNet要比Keras和Theano更像Python）。

## 2. Bi-LSTM条件随机场的讨论


后面小节暂时拖后翻译